In [1]:
import os
import gc

import pandas as pd
import gspread_dataframe as gs
import pyperclip as clip # copiar o texto clipboard
from time import sleep
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup

import random

# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/gdrive/projects/programming/automation")

#import sistemas
import sei.sei as sei 
import functions
import page
from page import *
import sei.sei_helpers as helpers

% reload_ext autoreload
% autoreload 2

In [2]:
gc = functions.authenticate_google("files/anatel.json")

In [8]:
PATH = os.path.abspath(r'S:\PUBLICO\Outorga 3º Andar\Provas de Radioamador\01.09\Classe B')

In [3]:
#PATH = os.path.abspath(r"C:\Users\rsilva\Desktop\SEI")
#HOME = os.path.abspath("C:/Users/rsilva/gdrive/projects/programming/automation/")

keys = ('Nome/Razão Social', 'Logradouro', 'Número', 'Complemento', 'Bairro', 'Cep', 'Município', 'UF', 'Número Fistel')

def string_endereço(dados):
    
    d = {}
    
    s = 'A(o)<br>' 
    
    s += dados["Nome/Razão Social"].title()
    
    s += '<br>' + dados["Logradouro"].title() + ", " + dados["Número"] + " " 
    
    s += dados["Complemento"].title() + " " 
    
    s += dados["Bairro"].title() + '<br>' 
    
    s += "CEP: " + dados["Cep"] + " - " + dados["Município"].title() + " - " + dados["UF"] 
            
    s += "<br><br>" + "<b>FISTEL: " + dados["Número Fistel"] + "</b>"
    
    d["A"] = s
    
    return d

def lastRow(ws, col=2):
    """ Find the last row in the worksheet that contains data.

    idx: Specifies the worksheet to select. Starts counting from zero.

    workbook: Specifies the workbook

    col: The column in which to look for the last cell containing data.
    """

    #ws = workbook.sheets[idx]

    lwr_r_cell = ws.cells.last_cell      # lower right cell
    lwr_row = lwr_r_cell.row             # row of the lower right cell
    lwr_cell = ws.range((lwr_row, col))  # change to your specified column

    if lwr_cell.value is None:
        lwr_cell = lwr_cell.end('up')    # go up untill you hit a non-empty cell

    return lwr_cell.row

In [4]:
usr = "rsilva"
pwd = "Savorthemom3nts"
sei_ = sei.login_sei(webdriver.Firefox(), usr, pwd)

In [34]:
sei_.itera_processos()

In [5]:
proc = "53504.006252/2018-75"

In [18]:
p = sei_.go_to_processo(proc)

In [13]:
files = sorted(os.listdir(PATH))
#file = os.path.join(PATH, files[-1])
files

['AGUITON NEVES.pdf',
 'ALEXANDRE MANTOANELLI.pdf',
 'ALEXANDRE TADEU MOLINA.pdf',
 'ALEXANDRE TURATI.pdf',
 'ALEXSANDRO CARVALHO DOS REIS.pdf',
 'AMAURI GERALDO GOMES DE ABREU.pdf',
 'ANDERSON ANDRADE ALENCAR.pdf',
 'ANDRE LUIZ MAFRA DE BEM.pdf',
 'ANTONIO CARLOS MARCOLA JUNIOR.pdf',
 'ANTONIO CORSI FERREIRA.pdf',
 'ARMANDO NERY SIGNORINI.pdf',
 'ARTHUR DE ARRUDA LEME DE VASCONCELLOS.pdf',
 'BENEDITO TEODORO.pdf',
 'BRUNO AZEREDO DE OLIVEIRA.pdf',
 'CARLOS ALBERTO DIAS FONTANA.pdf',
 'CARLOS APARECIDO GIOVANINI.pdf',
 'CARLOS ROBERTO GALHARDO.pdf',
 'CLAUDIO DE SOUZA FILHO.pdf',
 'DACIO FIGUEIREDO FILHO.pdf',
 'DAVID TREVISAN.pdf',
 'DAVILSON SALES.pdf',
 'DIEGO PADOVANI.pdf',
 'EDGARD PEREIRA TORRIANI.pdf',
 'EDSON BERTOLDO ALVES.pdf',
 'EGIDIO DRAUZIO DE MELO.pdf',
 'FABIO PASOTTI COLOZZA.pdf',
 'GABRIEL SPINDOLA RIBEIRO.pdf',
 'GLAYTON VINICIUS DAMASO.pdf',
 'GUILLERMO CREMERIUS.pdf',
 'HALEY MARCOS DE ALMEIDA.pdf',
 'HELDER VITOR TERRA.pdf',
 'HUGO BENEDITO MARCAL FERREIRA.pdf',
 

In [14]:
files.remove("Thumbs.db")

['AGUITON NEVES.pdf',
 'ALEXANDRE MANTOANELLI.pdf',
 'ALEXANDRE TADEU MOLINA.pdf',
 'ALEXANDRE TURATI.pdf',
 'ALEXSANDRO CARVALHO DOS REIS.pdf',
 'AMAURI GERALDO GOMES DE ABREU.pdf',
 'ANDERSON ANDRADE ALENCAR.pdf',
 'ANDRE LUIZ MAFRA DE BEM.pdf',
 'ANTONIO CARLOS MARCOLA JUNIOR.pdf',
 'ANTONIO CORSI FERREIRA.pdf',
 'ARMANDO NERY SIGNORINI.pdf',
 'ARTHUR DE ARRUDA LEME DE VASCONCELLOS.pdf',
 'BENEDITO TEODORO.pdf',
 'BRUNO AZEREDO DE OLIVEIRA.pdf',
 'CARLOS ALBERTO DIAS FONTANA.pdf',
 'CARLOS APARECIDO GIOVANINI.pdf',
 'CARLOS ROBERTO GALHARDO.pdf',
 'CLAUDIO DE SOUZA FILHO.pdf',
 'DACIO FIGUEIREDO FILHO.pdf',
 'DAVID TREVISAN.pdf',
 'DAVILSON SALES.pdf',
 'DIEGO PADOVANI.pdf',
 'EDGARD PEREIRA TORRIANI.pdf',
 'EDSON BERTOLDO ALVES.pdf',
 'EGIDIO DRAUZIO DE MELO.pdf',
 'FABIO PASOTTI COLOZZA.pdf',
 'GABRIEL SPINDOLA RIBEIRO.pdf',
 'GLAYTON VINICIUS DAMASO.pdf',
 'GUILLERMO CREMERIUS.pdf',
 'HALEY MARCOS DE ALMEIDA.pdf',
 'HELDER VITOR TERRA.pdf',
 'HUGO BENEDITO MARCAL FERREIRA.pdf',
 

In [25]:
for file in files[1:2]:
    p.incluir_doc_externo("Prova", os.path.join(PATH, file), arvore=file.split('.pdf')[0])
    p = sei_.go_to_processo(proc)

In [38]:
p.incluir_doc_externo("Solicitação", file, arvore="de Inscrição de Menor")

AttributeError: 'NavigableString' object has no attribute 'text'

In [ ]:
windows = sei_.driver.window_handles

sei_.driver.switch_to_window(windows[0])

sei_.driver.switch_to.default_content()